In [1]:
import pandas as pd
import numpy as np
from datetime import date

pd.set_option('display.max_columns', 500)

def print_full(x):
    try:
        v = len(x)
    except:
        v = 1000000
    pd.set_option('display.max_rows', v)
    print(x)
    pd.reset_option('display.max_rows')

In [2]:
file = open('uncleaned-data.csv', 'r', encoding='utf8')
df = pd.read_csv(file, dtype={
    "id": "string",
    "age": "string",
    "city": "string",
    "sex": "category",
    "province": "string",
    "country": "string",
    "latitude": float,
    "longitude": float,
    "geo_resolution": "category",
    "date_onset_symptoms": "string",
    "date_admission_hospital": "string",
    "date_confirmation": "string",
    "symptoms": "string",
    "travel_history_dates": "string",
    "travel_history_location": "string",
    "reported_market_exposure": "category",
    "additional_information": "string",
    "chronic_disease_binary": "string",
    "chronic_disease": "string",
    "source": "string",
    "sequence_available": "category",
    "outcome": "string",
    "date_death_or_discharge": "string",
    "notes_for_discussion": "string",
    "location": "string",
    "admin3": "string",
    "admin2": "string",
    "admin1": "string",
    "country_new": "string",
    "admin_id": "Int64",
    "data_moderator_initials": "string",
    "chronic_disease_binary": bool,
    "lives_in_Wuhan": "string",
    "travel_history_binary": "boolean"
})

In [ ]:
df['travel_history_binary'] = df['travel_history_binary'].fillna(value=False).astype(bool)

In [ ]:
df['lives_in_Wuhan'] = df['lives_in_Wuhan'].replace(['yes', 'no', pd.NA], [True, False, False]).astype(bool)

In [ ]:
df = df.drop(['ID', 'admin_id', 'source', 'admin1', 'admin2', 'admin3', 'data_moderator_initials', 'notes_for_discussion', 'location', 'country_new'], axis='columns')

# Remove columns that *could* be useful if the data was less sparse/more relevant to my project
df = df.drop(['date_death_or_discharge'], axis='columns')
df = df.drop(['additional_information'], axis='columns')
df = df.drop(['reported_market_exposure'], axis='columns')
df = df.drop(['city', 'province', 'country'], axis='columns')

In [6]:
def replace_all_with(df1, col, items, rep):
    df1[col] = df1[col].replace(items, [rep] * len(items))
    return df1

df = replace_all_with(df, 'outcome', ['recovered', 'Alive', 'not hospitalized', 'recovering at home 03.03.2020'], 'Recovered')
df = replace_all_with(df, 'outcome', ['Stable', 'Discharged', 'discharge', 'discharged', 'Discharged from hospital', 'Migrated', 'Migrated_Other', 'Symptoms only improved with cough. Currently hospitalized for follow-up.'], 'Recovered (hospitalized)')
df = replace_all_with(df, 'outcome', ['Death', 'Died', 'Dead', 'dead', 'death', 'died'], 'Deceased')
df = replace_all_with(df, 'outcome', ['https://www.mspbs.gov.py/covid-19.php', 'released from quarantine'], pd.NA)
df = replace_all_with(df, 'outcome', ['stable condition', 'stable', 'Under treatment', 'Receiving Treatment',], 'Hospitalized')
df = replace_all_with(df, 'outcome', ['severe', 'unstable', 'severe illness', 'critical condition, intubated as of 14.02.2020', 'critical condition', 'treated in an intensive care unit (14.02.2020)'], 'Critical condition')

df = df.dropna(how='any', subset=['outcome'])

df['outcome'] = df['outcome'].astype('category')
category_dict = dict(enumerate(df['outcome'].cat.categories))
df['outcome'] = df['outcome'].cat.codes

In [7]:
category_dict

{0: 'Critical condition',
 1: 'Deceased',
 2: 'Hospitalized',
 3: 'Recovered',
 4: 'Recovered (hospitalized)'}

In [ ]:
df['sequence_available_binary'] = df['sequence_available'].replace([pd.NA, 'yes, BetaCoV/Mexico/CDMX/InDRE_01/2020', 'yes', '28.02.2020', '02.03.2020', '10.03.2020'], [False, True, True, True, True, True]).astype(bool)

In [ ]:
df['sex'] = df['sex'].cat.codes
df['geo_resolution'] = df['geo_resolution'].cat.codes
df['sequence_available'] = df['sequence_available'].cat.codes

In [ ]:
df['age'] = df['age'].str.replace(r'(\d+) (month|week)s?', '0', regex=True)
df = df.combine_first(df['age'].str.extract(r'^(?P<age_min>\d?\.?\d+)\s*-?\s*(?P<age_max>\d+)?$')).drop(['age'], axis=1)

In [ ]:
df['age_max'] = np.where(df['age_max'].isnull(), df['age_min'], df['age_max'])

df = df.fillna(value={
    'age_min': '-1',
    'age_max': '-1'
})

df['age_min'] = df['age_min'].astype('float')
df['age_max'] = df['age_max'].astype('float')

In [ ]:
def convert_col_to_days_since(series, since, invert=False, impute= -1):
    if not invert:
        v = series - since
    else:
        v = since - series
    
    v = v.dt.days
    v = v.fillna(value=impute)
    return v

In [ ]:
df = df.combine_first(df['date_confirmation'].str.extract(r'^(?P<date_confirmation_min>\d+\.\d+\.\d+)\s*-?\s*(?P<date_confirmation_max>\d+\.\d+\.\d+)?$')).drop(['date_confirmation'], axis=1)
df['date_confirmation_max'] = np.where(df['date_confirmation_max'].isnull(), df['date_confirmation_min'], df['date_confirmation_max'])

In [ ]:
df[['date_confirmation_min', 'date_confirmation_max']] = df[['date_confirmation_min', 'date_confirmation_max']].apply(pd.to_datetime)

df['ds_date_confirmation_min'] = convert_col_to_days_since(df['date_confirmation_min'], df['date_confirmation_min'].min())
df['ds_date_confirmation_max'] = convert_col_to_days_since(df['date_confirmation_max'], df['date_confirmation_min'].min())

In [ ]:
#df[~pd.isnull(df['date_onset_symptoms'])][['date_confirmation', 'date_onset_symptoms']]

df['date_onset_symptoms'] = df['date_onset_symptoms'].replace(['01.01.2020-12.01.2020'], ['06.01.2020'])
df['date_onset_symptoms'] = pd.to_datetime(df['date_onset_symptoms'])

df['ds_date_onset_symptoms'] = convert_col_to_days_since(df['date_onset_symptoms'], df['date_confirmation_min'], invert=True, impute=0)
df = df.drop(['date_onset_symptoms'], axis=1)

In [ ]:
df['date_admission_hospital'] =  pd.to_datetime(df['date_admission_hospital'])

df['ds_date_admission_hospital'] = convert_col_to_days_since(df['date_admission_hospital'], df['date_confirmation_min'], invert=True, impute=0)
df = df.drop(['date_admission_hospital'], axis=1)

In [ ]:
df = df.drop(['date_confirmation_min', 'date_confirmation_max'], axis=1)

In [ ]:
df = pd.concat([df, df['travel_history_dates'].str.extract(r'(?P<date_enter_Wuhan>\d+\.\d+\.\d+)?\s*-?\s*(?=(?P<date_exit_Wuhan>\d+\.\d+\.\d+))')], axis=1)
df = df.drop(['travel_history_dates'], axis=1)

df['date_enter_Wuhan'] = pd.to_datetime(df['date_enter_Wuhan'])
df['date_exit_Wuhan'] = pd.to_datetime(df['date_exit_Wuhan'])
df['time_in_Wuhan'] = abs((df[~pd.isnull(df['date_exit_Wuhan'])]['date_exit_Wuhan'] - df['date_enter_Wuhan']).dt.days)
df['time_in_Wuhan'] = np.where(pd.isnull(df['time_in_Wuhan']) & ~pd.isnull(df['date_exit_Wuhan']), df['time_in_Wuhan'].mean(), df['time_in_Wuhan'])

df['time_in_Wuhan'] = df['time_in_Wuhan'].fillna(value=0)
df = df.drop(['date_enter_Wuhan', 'date_exit_Wuhan'], axis=1)

In [ ]:
# Drop a row that contains a location under chronic_disease since it's likely bad data entry
df = df[~df['chronic_disease'].str.contains(r'Iran', na=False)]

In [ ]:
df['chronic_disease'] = df['chronic_disease'].str.lower()
df['chronic_disease'] = df['chronic_disease'].str.replace(r'(:|;)', ",", regex=True)
df['chronic_disease'] = df['chronic_disease'].str.replace(r'(history of hypertension|hypertension for more than 20 years|hypertenstion|hypertensive)', 'hypertension', regex=True)
df['chronic_disease'] = df['chronic_disease'].str.replace('copd', 'chronic obstructive pulmonary disease')
df['chronic_disease'] = df['chronic_disease'].str.replace('diabetes for more than 20 years', 'diabetes')

df = pd.concat([df, df['chronic_disease'].str.get_dummies(sep=',').add_prefix('chronic_disease_')], axis=1).drop('chronic_disease', axis=1)

In [ ]:
df['symptoms'] = df['symptoms'].str.lower()
df['symptoms'] = df['symptoms'].str.replace(r'(:|;)', ",", regex=True)
df = df[~df['symptoms'].str.contains('none', na=False)]

df = pd.concat([df, df['symptoms'].str.get_dummies(sep=',').add_prefix('symptoms_')], axis=1).drop('symptoms', axis=1)

In [ ]:
df['travel_history_location'] = df['travel_history_location'].str.lower()
df['travel_history_location'] = df['travel_history_location'].str.replace(r'(:|;)', ",", regex=True)
df = pd.concat([df, df['travel_history_location'].str.get_dummies(sep=',').add_prefix('travel_history_includes_')], axis=1).drop('travel_history_location', axis=1)

In [26]:
df.to_pickle('./processed-data.pkl')
df.to_csv('./processed-data.csv', index=False)